# Create grids for double-gyre simulation 
Code creates a 'rectangular' (in spherical projection) grid bounded by walls for an idealized double gyre setup. Only the hgrid, topog, and geothermal flux files are created; FMS is used to create land and ocean masks.

Current ocean boundaries with horizontal resolution of 1&deg;, 1/9&deg;, and 1/27&deg;:
```
southlat  = 20   # southern latitude
lenlat    = 40   # latitudinal length of domain
westlon   = -55  # western longitude
lenlon    = 40   # longitudinal length of domain
min_depth = 0
max_depth = 4000  
```

### Notes:
- The geothermal file can be used as input without further modification.
- Due to an impenetrable formating error, however, `hgrid.nc` and `topog.nc` files need to be ran through Enhui's Matlab function before using FMS to generate all other necessary grid files.

### Troubleshooting notes:
In case anyone else wants to try to solve the issue of not being able to create `hgrid.nc` and `topog.nc` files that work with FMS, here is what I tried. Note that the next logical step is to use `gridtools` but I have not done so.
- `hgrid.nc` seems to work for make_solo_mosaic but `topog.nc` does not seem to work with make_quick_mosaic (`mking3_mask_mosaic.sh`)
- Tried a range of fill values, including -1e+20 and np.NaN because Enhui's Matlab script uses NaN as fill value.
- Tried a range of netcdf format, including `NETCDF4_CLASSIC` (as with Enhui's files), and `NETCDF3_64BIT_OFFSET` (following Raphael's recommendation). 
- Netcdf files appear identical to Enhui's but still no luck.

In [2]:
import os
import numpy as np
import xarray as xr
import numpy.matlib
import netCDF4 as nc
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
# Function to calculate distance between two lat/lon point (using sphere approximation)
from math import radians, degrees, sin, cos, asin, acos, sqrt

def great_circle(lon1, lat1, lon2, lat2, flat = 1):
    
    #if flat:
    mlat = 85000 #°deg lat in m
    dist = np.sqrt( ((lon1-lon2)*mlat)**2 + ((lat1-lat2)*mlat)**2 )
    
    #else:
    #    R = 6367442.76
    #    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])    
    #    dist = R * (acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2)) )
    
    return dist

# Interestingly, in this function lat/lon 1 and 2 are interchangeable 
# but in spheric_dist it causes some discrepency
# answer very similar though

In [4]:
# Size of ocean basin (must add one cell on each side for land)
southlat  = 20   # southern latitude
lenlat    = 40   # latitudinal length of domain
westlon   = -55  # western longitude
lenlon    = 40   # longitudinal length of domain
min_depth = 0
max_depth = 4000  

# Specify resolution - dx, dy needed for hgrid
dx_in     = 1/27
dy_in     = dx_in   # in our case dy = dx

# Save paths - depends on resolution
spath = '../'
if (dx_in == 1.0):
    spathTopog = spath + 'DG_1deg/DG_topog_1deg_py.nc'
    spathHgrid = spath + 'DG_1deg/DG_hgrid_1deg_py.nc'
    spathGeoth = spath + 'DG_1deg/DG_geothermal0_1deg_py.nc'
elif (dx_in == 1/9):
    spathTopog = spath + 'DG_011deg/DG_topog_011deg_py.nc'
    spathHgrid = spath + 'DG_011deg/DG_hgrid_011deg_py.nc'
    spathGeoth = spath + 'DG_011deg/DG_geothermal0_011deg_py.nc'
elif (dx_in == 1/27):
    spathTopog = spath + 'DG_0037deg/DG_topog_0037deg_py.nc'
    spathHgrid = spath + 'DG_0037deg/DG_hgrid_0037deg_py.nc'
    spathGeoth = spath + 'DG_0037deg/DG_geothermal0_0037deg_py.nc'
    
# Fill value
fillval   = np.NaN # -1e+20  # Tried both values; Enhui's files have NaN...

# Calculate ni & nj
ni        = lenlon/dx_in+1+2  # number of grid points in x-direction (2 cells for land)
nj        = lenlat/dy_in+1+2  # number of grid points in y-direction (2 cells for land)

In [5]:
# Design coordinates
# q pts are at the edge points
xq = np.linspace(westlon-dx_in, westlon+lenlon+dx_in, int(ni)).reshape(1, int(ni))
yq = np.linspace(southlat-dy_in, southlat+lenlat+dy_in, int(nj)).reshape(int(nj),1)

# h pts are in the middle
xh = np.linspace(westlon-(dx_in/2), westlon+lenlon+(dx_in/2), int(ni)-1).reshape(1, int(ni)-1)
yh = np.linspace(southlat-(dy_in/2), southlat+lenlat+(dy_in/2), int(nj)-1).reshape(int(nj)-1, 1)

In [6]:
# hgrid file requires the edges and mid-points
nxp = np.arange(1, 2*ni)
nyp = np.arange(1, 2*nj)
nx  = np.arange(1, len(nxp))
ny  = np.arange(1, len(nyp))
x   = np.matlib.repmat(np.linspace(westlon-dx_in, westlon+lenlon+dx_in, 2*int(ni)-1), len(nyp), 1)
y1  = np.linspace(southlat-dy_in, southlat+lenlat+dy_in, 2*int(nj)-1)
y   = np.matlib.repmat(y1.reshape(len(nyp), 1), 1, len(nxp))
y   = y.reshape(len(nyp), len(nxp))

# Non-rotated grid
angle_dx = np.matlib.repmat(float(0), len(nyp), len(nxp))

# Confirmed that great_circle gives the same answer as spheric_dist()

# Calculate dx - do so through each row
dx  = np.full((len(nyp), len(nx)), float(0))
for irow in np.arange(0,len(nyp)):
    for ival in np.arange(0, len(nx)):
        dx[irow,ival] = great_circle(x[irow,ival], y[irow,ival],
                                         x[irow,ival+1], y[irow,ival+1])
        
# Calculate dy - do so through each column
dy  = np.full((len(ny), len(nxp)), float(0))
for icol in np.arange(0,len(nxp)):
    for ival in np.arange(0, len(ny)):
        dy[ival,icol] = great_circle(x[ival,icol], y[ival,icol],
                                         x[ival+1,icol], y[ival+1,icol])

# Confirmed that dy is the same as before        
area = dx[:-1,:] * dy[:,:-1]

In [7]:
# Convert any NaN into fillval
x[np.isnan(x)]               = fillval
y[np.isnan(y)]               = fillval
dx[np.isnan(dx)]             = fillval
dy[np.isnan(dy)]             = fillval
area[np.isnan(area)]         = fillval
angle_dx[np.isnan(angle_dx)] = fillval

In [8]:
### Saving using netcdf4 because can't get xarray to work
writing = nc.Dataset(spathHgrid, "w", format="NETCDF4_CLASSIC")
NXP     = writing.createDimension("nxp", len(nxp))
NYP     = writing.createDimension("nyp", len(nyp))  
NX      = writing.createDimension("nx", len(nx))
NY      = writing.createDimension("ny", len(ny))  
string  = writing.createDimension("string", 255)

#NXP                = writing.createVariable("nxp","f8",("nxp"),fill_value=fillval)
NXP                = writing.createVariable("nxp","f8",("nxp"))
NXP[:]             = nxp
NXP.standard_name  = "longitude"
NXP.long_name      = "longitude"
NXP.units          = "degrees_east"
NXP.axis           = "X"

#NYP                = writing.createVariable("nyp","f8",("nyp"),fill_value=fillval)
NYP                = writing.createVariable("nyp","f8",("nyp"))
NYP[:]             = nyp
NYP.standard_name  = "latitude"
NYP.long_name      = "latitude"
NYP.units          = "degrees_north"
NYP.axis           = "Y"

#NX                 = writing.createVariable("nx","f8",("nx"),fill_value=fillval)
NX                 = writing.createVariable("nx","f8",("nx"))
NX[:]              = nx
NX.standard_name   = "longitude"
NX.long_name       = "longitude"
NX.units           = "degrees_east"
NX.axis            = "X"

#NY                 = writing.createVariable("ny","f8",("ny"),fill_value=fillval)
NY                 = writing.createVariable("ny","f8",("ny"))
NY[:]              = ny
NY.standard_name   = "latitude"
NY.long_name       = "latitude"
NY.units           = "degrees_north"
NY.axis            = "Y"

X                  = writing.createVariable("x","f8",("nyp", "nxp"),fill_value=fillval)
X[:,:]             = x
X.long_name        = "super grid x"
X.units            = "degree"

Y                  = writing.createVariable("y","f8",("nyp", "nxp"),fill_value=fillval)
Y[:,:]             = y
Y.long_name        = "super grid y"
Y.units            = "degree"

DX                 = writing.createVariable("dx","f8",("nyp", "nx"),fill_value=fillval)
DX[:,:]            = dx
DX.long_name       = "super grid length dx"
DX.units           = "meter"

DY                 = writing.createVariable("dy","f8",("ny", "nxp"),fill_value=fillval)
DY[:,:]            = dy
DY.long_name       = "super grid length dy"
DY.units           = "meter"

AREA               = writing.createVariable("area","f8",("ny", "nx"),fill_value=fillval)
AREA[:,:]          = area
AREA.long_name     = "super grid area"
AREA.units         = "m2"

ANGLE_DX           = writing.createVariable("angle_dx","f8",("nyp", "nxp"),fill_value=fillval)
ANGLE_DX[:,:]      = angle_dx
ANGLE_DX.long_name = "grid cell angle of xi and east"
ANGLE_DX.units     = "degree"

TILE               = writing.createVariable("tile","S1",("string"))
TILE[:]            = 'tile'

# Global attributes
writing.creator       = "MPO in LRG at Princeton University"
writing.creation_date = datetime.now().strftime("%d-%b-%Y %H:%M:%S")

writing.close()

In [9]:
# Topog file requires mid-points
nx = np.arange(1, xh.size+1)
ny = np.arange(1, yh.size+1)

# Set depth to max_depth everywhere, except the outside grids 
depth = np.matlib.repmat(max_depth, len(ny), len(nx))
depth[0,:]  = min_depth
depth[:,0]  = min_depth
depth[-1,:] = min_depth
depth[:,-1] = min_depth

# Variance of sub-grid scale topography
h2 = np.matlib.repmat(0, len(ny), len(nx))

# Wet is mask with 1 = ocean
wet = np.matlib.repmat(1, len(ny), len(nx))
wet[0,:]  = 0
wet[:,0]  = 0
wet[-1,:] = 0
wet[:,-1] = 0

# Not sure what modified_mask is so set equal to wet -- not included in file
modified_mask = wet

In [10]:
### Saving using netcdf4 because can't get xarray to work
writing = nc.Dataset(spathTopog, "w", format="NETCDF4_CLASSIC")
NX      = writing.createDimension("nx", len(nx))
NY      = writing.createDimension("ny", len(ny))  
ntiles  = writing.createDimension("ntiles", 1)

#NX                 = writing.createVariable("nx","f8",("nx"),fill_value=fillval)
NX                 = writing.createVariable("nx","f8",("nx"))
NX[:]              = nx
NX.standard_name   = "longitude"
NX.long_name       = "longitude"
NX.units           = "degrees_east"
NX.axis            = "X"

#NY                 = writing.createVariable("ny","f8",("ny"),fill_value=fillval)
NY                 = writing.createVariable("ny","f8",("ny"))
NY[:]              = ny
NY.standard_name   = "latitude"
NY.long_name       = "latitude"
NY.units           = "degrees_north"
NY.axis            = "Y"

DEPTH              = writing.createVariable("depth","f8",("ny", "nx"),fill_value=fillval)
DEPTH[:,:]         = depth
DEPTH.long_name    = "topographic depth at T-cell centers"
DEPTH.units        = "meter"

H2                 = writing.createVariable("h2","f8",("ny", "nx"),fill_value=fillval)
H2[:,:]            = h2
H2.long_name       = "variance of sub-grid scale topography"
H2.units           = "meter"

WET                = writing.createVariable("wet","f8",("ny", "nx"),fill_value=fillval)
WET[:,:]           = wet
WET.long_name      = "land=0 ocean=1 mask"
WET.units          = "none"

NTILES             = writing.createVariable("ntiles","f8",("ntiles"))
NTILES[:]          = 1

# Global attributes
writing.creator       = "MPO in LRG at Princeton University"
writing.creation_date = datetime.now().strftime("%d-%b-%Y %H:%M:%S")

writing.close()

In [11]:
### Create a geothermal file of the same size as the grid
# Filled with zero for idealized simulation 
# If actual values were needed, MOM6-examples has code to regrid the davies geothermal file
writing = nc.Dataset(spathGeoth, "w", format="NETCDF4_CLASSIC")
lat      = writing.createDimension("lat", len(ny))
lon      = writing.createDimension("lon", len(nx))  

lat                 = writing.createVariable("lat","f8",("lat"),fill_value=np.NaN)
lat[:]              = ny
lat.units           = "degrees_north"
lat.axis            = "Y"

lon                 = writing.createVariable("lon","f8",("lon"),fill_value=np.NaN)
lon[:]              = nx
lon.units           = "degrees_east"
lon.axis            = "X"

geothermal_hf            = writing.createVariable("geothermal_hf","f8",("lat", "lon"),fill_value=fillval)
geothermal_hf[:,:]       = h2
geothermal_hf.long_name  = "Geothermal heat flow"
geothermal_hf.units      = "w/m2"

# Global attributes
writing.creator       = "MPO in LRG at Princeton University"
writing.creation_date = datetime.now().strftime("%d-%b-%Y %H:%M:%S")

writing.close()

In [12]:
np.shape(area)

(2164, 2164)